# Herschel Stripe 82 master catalogue
## Preparation of DES data

Blanco DES catalogue: the catalogue comes from `dmu0_DES`.

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position;
- The G band stellarity;
- The magnitude for each band.
- The auto/kron magnitudes/fluxes to be used as total magnitude.
- The aperture magnitudes, which are used to compute a corrected 2 arcsec aperture magnitude.

We don't know when the maps have been observed. We will take the final observation date as 2017.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
0246c5d (Thu Jan 25 17:01:47 2018 +0000) [with local modifications]
This notebook was executed on: 
2018-02-23 11:53:49.943241


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates, \
                                            nb_plot_mag_ap_evol, nb_plot_mag_vs_apcor 
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux, flux_to_mag, aperture_correction

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [3]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "des_ra"
DEC_COL = "des_dec"

In [4]:
# Pristine HSC catalogue
orig_des = Table.read("../../dmu0/dmu0_DES/data/DES-DR1_Herschel-Stripe-82.fits")

FileNotFoundError: [Errno 2] No such file or directory: '../../dmu0/dmu0_DES/data/DES-DR1_Herschel-Stripe-82.fits'

## 1 - Aperture correction

To compute aperture correction we need to dertermine two parametres: the target aperture and the range of magnitudes for the stars that will be used to compute the correction.

Target aperture: To determine the target aperture, we simulate a curve of growth using the provided apertures and draw two figures:

The evolution of the magnitudes of the objects by plotting on the same plot aperture number vs the mean magnitude.
The mean gain (loss when negative) of magnitude is each aperture compared to the previous (except for the first of course).
As target aperture, we should use the smallest (i.e. less noisy) aperture for which most of the flux is captures.

Magnitude range: To know what limits in aperture to use when doing the aperture correction, we plot for each magnitude bin the correction that is computed and its RMS. We should then use the wide limits (to use more stars) where the correction is stable and with few dispersion.

In [ ]:
bands = ["g", "r", "i", "z", "y"]
apertures = ["1", "2", "3", "4",  "5", "6", "7", "8", "9", "10", "11"] #Removed "40" and "235" because they lack errors

magnitudes = {}
stellarities = {}

for band in bands:
    magnitudes[band] = np.array(
        [orig_des["MAG_APER_{}_{}".format(aperture, band.upper())] for aperture in apertures]
    )

    stellarities[band] = np.array(orig_des["CLASS_STAR_{}".format(band.upper())])
    
    # Some sources have an infinite magnitude
    mask = np.isclose(magnitudes[band], 99.)
    magnitudes[band][mask] = np.nan

    
mag_corr = {}

### I.a - g band

In [ ]:

nb_plot_mag_ap_evol(magnitudes['g'], stellarities['g'], labels=apertures)

We will use aperture 10 as target.

In [ ]:
nb_plot_mag_vs_apcor(orig_des['MAG_APER_4_G'], orig_des['MAG_APER_10_G'], stellarities['g'])


We will use magnitudes between 15.0 and 16.0.

In [ ]:
# Aperture correction
mag_corr['g'], num, std = aperture_correction(
    orig_des['MAG_APER_4_G'], orig_des['MAG_APER_10_G'], 
    stellarities['g'],
    mag_min=15.0, mag_max=16.0)
print("Aperture correction for g band:")
print("Correction: {}".format(mag_corr['g']))
print("Number of source used: {}".format(num))
print("RMS: {}".format(std))

### I.b - r band

In [ ]:

nb_plot_mag_ap_evol(magnitudes['r'], stellarities['r'], labels=apertures)

We will use aperture 10 as target.

In [ ]:

nb_plot_mag_vs_apcor(orig_des['MAG_APER_4_R'], orig_des['MAG_APER_10_R'], stellarities['r'])

We use magnitudes between 15.0 and 16.0.

In [ ]:
# Aperture correction
mag_corr['r'], num, std = aperture_correction(
    orig_des['MAG_APER_4_R'], orig_des['MAG_APER_10_R'], 
    stellarities['r'],
    mag_min=15.0, mag_max=16.0)
print("Aperture correction for r band:")
print("Correction: {}".format(mag_corr['r']))
print("Number of source used: {}".format(num))
print("RMS: {}".format(std))

### I.b - i band

In [ ]:
nb_plot_mag_ap_evol(magnitudes['i'], stellarities['i'], labels=apertures)

We will use aperture 10 as target.

In [ ]:
nb_plot_mag_vs_apcor(orig_des['MAG_APER_4_I'], orig_des['MAG_APER_10_I'], stellarities['i'])

We use magnitudes between 15.0 and 16.0.

In [ ]:
# Aperture correction
mag_corr['i'], num, std = aperture_correction(
    orig_des['MAG_APER_4_I'], orig_des['MAG_APER_10_I'], 
    stellarities['i'],
    mag_min=15.0, mag_max=16.0)
print("Aperture correction for i band:")
print("Correction: {}".format(mag_corr['i']))
print("Number of source used: {}".format(num))
print("RMS: {}".format(std))

### I.b - z band

In [ ]:

nb_plot_mag_ap_evol(magnitudes['z'], stellarities['z'], labels=apertures)

We will use aperture 57 as target.

In [ ]:

nb_plot_mag_vs_apcor(orig_des['MAG_APER_4_Z'], orig_des['MAG_APER_10_Z'], stellarities['z'])

We use magnitudes between 15.0 and 16.0.

In [ ]:
# Aperture correction
mag_corr['z'], num, std = aperture_correction(
    orig_des['MAG_APER_4_Z'], orig_des['MAG_APER_10_Z'], 
    stellarities['z'],
    mag_min=15.0, mag_max=16.0)
print("Aperture correction for z band:")
print("Correction: {}".format(mag_corr['z']))
print("Number of source used: {}".format(num))
print("RMS: {}".format(std))

### I.b - y band

In [ ]:
nb_plot_mag_ap_evol(magnitudes['y'], stellarities['y'], labels=apertures)

We will use aperture 10 as target.

In [ ]:

nb_plot_mag_vs_apcor(orig_des['MAG_APER_4_Y'], orig_des['MAG_APER_10_Y'], stellarities['y'])


We use magnitudes between 15.0 and 16.0.

In [ ]:
# Aperture correction
mag_corr['y'], num, std = aperture_correction(
    orig_des['MAG_APER_4_Y'], orig_des['MAG_APER_10_Y'], 
    stellarities['y'],
    mag_min=15.0, mag_max=16.0)
print("Aperture correction for y band:")
print("Correction: {}".format(mag_corr['y']))
print("Number of source used: {}".format(num))
print("RMS: {}".format(std))

## 2 - Column selection

In [ ]:

imported_columns = OrderedDict({
        'COADD_OBJECT_ID': "des_id",
        'RA': "des_ra",
        'DEC': "des_dec",
        'CLASS_STAR_G':  "des_stellarity",
    
        'MAG_AUTO_G': "m_decam_g", 
        'MAGERR_AUTO_G': "merr_decam_g", 
        'MAG_APER_4_G': "m_ap_decam_g", 
        'MAGERR_APER_4_G': "merr_ap_decam_g",
    
        'MAG_AUTO_R': "m_decam_r", 
        'MAGERR_AUTO_R': "merr_decam_r", 
        'MAG_APER_4_R': "m_ap_decam_r", 
        'MAGERR_APER_4_R': "merr_ap_decam_r",
    
        'MAG_AUTO_I': "m_decam_i", 
        'MAGERR_AUTO_I': "merr_decam_i", 
        'MAG_APER_4_I': "m_ap_decam_i", 
        'MAGERR_APER_4_I': "merr_ap_decam_i",
    
        'MAG_AUTO_Z': "m_decam_z", 
        'MAGERR_AUTO_Z': "merr_decam_z", 
        'MAG_APER_4_Z': "m_ap_decam_z", 
        'MAGERR_APER_4_Z': "merr_ap_decam_z",
    
        'MAG_AUTO_Y': "m_decam_y", 
        'MAGERR_AUTO_Y': "merr_decam_y", 
        'MAG_APER_4_Y': "m_ap_decam_y", 
        'MAGERR_APER_4_Y': "merr_ap_decam_y",

    })


catalogue = Table.read("../../dmu0/dmu0_DES/data/DES-DR1_Herschel-Stripe-82.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]

epoch = 2017

# Clean table metadata
catalogue.meta = None

In [ ]:
# Aperture correction
for band in bands:
    catalogue["m_ap_decam_{}".format(band)] += mag_corr[band]

In [ ]:
# Adding flux and band-flag columns
for col in catalogue.colnames:

            
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        
        # Some objects have -99.0 values
        mask = (catalogue[col] > 50.) | (catalogue[col] < 0.)
        catalogue[col][mask] = np.nan
        catalogue[errcol][mask] = np.nan
        
        flux, error = mag_to_flux(np.array(catalogue[col]), np.array(catalogue[errcol]))
            
        # Fluxes are added in µJy
        catalogue.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        catalogue.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        if 'ap' not in col:
            catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.            
        


In [ ]:
catalogue[:10].show_in_notebook()

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [ ]:
SORT_COLS = ['merr_ap_decam_g', 'merr_ap_decam_r','merr_ap_decam_i','merr_ap_decam_z','merr_ap_decam_y']
FLAG_NAME = 'des_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(catalogue, RA_COL, DEC_COL, sort_col=SORT_COLS,flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_Herschel-Stripe-82.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec,  near_ra0=True)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
catalogue[RA_COL] = catalogue[RA_COL] + delta_ra.to(u.deg)
catalogue[DEC_COL] = catalogue[DEC_COL] + delta_dec.to(u.deg)

In [ ]:
catalogue[RA_COL].unit = u.deg
catalogue[DEC_COL].unit = u.deg

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec,  near_ra0=True)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "des_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

## V - Flagging objects near bright stars

# VI - Saving to disk

In [ ]:
catalogue.write("{}/DES.fits".format(OUT_DIR), overwrite=True)